## lenstronomy lens equation solver and flux ratios computation example notebook

In [37]:
# import standard python modules
import copy
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable
%matplotlib inline


# import the lens model class 
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
# import the lens equation solver class (finding image plane positions of a source position)
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
# import lens model solver with 4 image positions constrains
from lenstronomy.LensModel.Solver.solver4point import Solver4Point #Solver4Point
from lenstronomy.LensModel.Solver.solver2point import Solver2Point #Solver4Point

In [38]:
# chose a lens model (list of parameterized lens models)
lens_model_list = ['PEMD', 'SHEAR_GAMMA_PSI'] #['EPL'] #, 'SHEAR_GAMMA_PSI'] #, 'NFW', 'NFW']

#initialize classes with the specific lens models
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)
solver2Point = Solver2Point(lensModel=lensModel, solver_type='THETA_E_ELLIPSE')
solver4Point = Solver4Point(lensModel=lensModel, solver_type='PROFILE_SHEAR')

In [39]:
# DCLS1507+0522

# # x_image, y_image F200LP known values
# x_image_F140W = [2.1056694526151, -2.1446663684802947]
# y_image_F140W = [0.16656821131187893, -0.09666993968892257]

# kwargs_lens_init_F140W = [{'theta_E': 4.923897427699637, 
#                             'gamma': 2.0, 
#                             'e1': -0.20087942999736097, 
#                             'e2': 0.05268333713312042, 
#                             'center_x': 0.0038999843035251564, 
#                             'center_y': 0.00931611330505573}]

# kwargs_fit_F140W, precision_F140W = solver2Point.constraint_lensmodel(x_pos=x_image_F140W, y_pos=y_image_F140W, 
#                                                           kwargs_list=kwargs_lens_init_F140W, xtol=1.49012e-10)

# x_source_new_F140W, y_source_new_F140W = lensModel.ray_shooting(x_image_F140W, y_image_F140W, kwargs_fit_F140W)
# print('F140W source center:', x_source_new_F140W, y_source_new_F140W)
# print('F140W kwargs_fit:', kwargs_fit_F140W)

# x_image, y_image F200LP known values
# Image 1: 1.2285433511852109 4.1607764484457945
# Image 2: -1.3986311119335793 4.312169573523052
# Image 3: -3.6291087887239137 3.4710665194784394
# Image 4: -0.8344498087021135 -1.9958577504452544

x_image_F200LP = np.array([1.2285433511852109, -1.3986311119335793, -3.6291087887239137, -0.8344498087021135])
y_image_F200LP = np.array([4.1607764484457945, 4.312169573523052, 3.4710665194784394, -1.9958577504452544])

# # DCLS1507+0522_local_customlogL_fitF200LP_s1_run_11
kwargs_pemd = {'theta_E': 4.923897427699637, 
                 'gamma': 2.0, 
                 'e1': -0.20087942999736097, 
                 'e2': 0.05268333713312042, 
                 'center_x': 0.0038999843035251564, 
                 'center_y': 0.00931611330505573}
kwargs_shear = {'gamma_ext': -0.001807834284974115, 
                  'psi_ext': -0.02465806220709, 
                  'ra_0': 0, 
                  'dec_0': 0}

# # DCLS1507+0522_local_customlogL_fitF200LP_s1_run_13
# kwargs_pemd = {'theta_E': 4.869725983283605, 
#                'gamma': 2.0, 
#                'e1': -0.2190712079617276, 
#                'e2': 0.03047620490953712, 
#                'center_x': 0.0027477505460033825, 
#                'center_y': 0.01223789522781755}
# kwargs_shear = {'gamma_ext': -0.05593647005675678, 
#                 'psi_ext': 0.030788955760211307, 
#                 'ra_0': 0, 
#                 'dec_0': 0}

kwargs_lens_list = [kwargs_pemd, kwargs_shear]

kwargs_lens_init_F200LP = copy.deepcopy(kwargs_lens_list)

kwargs_lens_init_F200LP[0] = {'theta_E': 4.923897427699637, 
                            'gamma': 2.0, 
                            'e1': -0.20087942999736097, 
                            'e2': 0.05268333713312042, 
                            'center_x': 0.0038999843035251564, 
                            'center_y': 0.00931611330505573}

# kwargs_fit_F200LP, precision_F200LP = solver2Point.constraint_lensmodel(x_pos=x_image_F200LP, y_pos=y_image_F200LP, 
#                                                           kwargs_list=kwargs_lens_init_F200LP, xtol=1.49012e-10)

# x_source_new_F200LP, y_source_new_F200LP = lensModel.ray_shooting(x_image_F200LP, y_image_F200LP, kwargs_fit_F200LP)
# print('F200LP source center:', x_source_new_F200LP, y_source_new_F200LP)
# print('F200LP kwargs_fit:', kwargs_fit_F200LP)

#############################################################################
#############################################################################

x_image, y_image = x_image_F200LP[0], y_image_F200LP[0]

alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_lens_list)

x_source, y_source = x_image-alpha_x, y_image-alpha_y

print(x_source, y_source)

x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source, sourcePos_y=y_source, 
                                                                 min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
print('image positions: ', x_image, y_image)

kwargs_fit_F200LP, precision_F200LP = solver2Point.constraint_lensmodel(x_pos=x_image, y_pos=y_image, 
                                                          kwargs_list=kwargs_lens_init_F200LP, xtol=1.49012e-10)

x_source_new_F200LP, y_source_new_F200LP = lensModel.ray_shooting(x_image, y_image, kwargs_fit_F200LP)
print('F200LP source center:', x_source_new_F200LP, y_source_new_F200LP)
print('F200LP kwargs_fit:', kwargs_fit_F200LP)

-0.28503264215938806 -0.13189737927927858
image positions:  [4.68951229 0.34106153 1.22854335] [-1.55409092 -4.65686554  4.16077645]
F200LP source center: [-0.28503264 -0.28503264 -0.28503264] [-0.13189738 -0.13189738 -0.13189738]
F200LP kwargs_fit: [{'theta_E': 4.923897427698462, 'gamma': 2.0, 'e1': -0.20087942999743402, 'e2': 0.05268333713284188, 'center_x': 0.0038999843035251564, 'center_y': 0.00931611330505573}, {'gamma_ext': -0.001807834284974115, 'psi_ext': -0.02465806220709, 'ra_0': 0, 'dec_0': 0}]


In [59]:
kwargs_fit_test, precision_test = solver4Point.constraint_lensmodel(x_pos=x_image_F200LP, y_pos=y_image_F200LP, 
                                                          kwargs_list=kwargs_lens_list, xtol=1.49012e-12)

In [60]:
kwargs_fit_test

[{'theta_E': 3.5823362780010566,
  'gamma': 2.0,
  'e1': -0.18424270041316954,
  'e2': 0.022617353408534314,
  'center_x': -0.6389054975957009,
  'center_y': 0.500032208309703},
 {'gamma_ext': -0.001807834284974115,
  'psi_ext': 627.2785003856497,
  'ra_0': 0,
  'dec_0': 0}]

In [61]:
x_image_F200LP, y_image_F200LP

(array([ 1.22854335, -1.39863111, -3.62910879, -0.83444981]),
 array([ 4.16077645,  4.31216957,  3.47106652, -1.99585775]))

In [62]:
x_image, y_image = x_image_F200LP[0], y_image_F200LP[0]

alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_fit_test)

x_source, y_source = x_image-alpha_x, y_image-alpha_y

print(x_source, y_source)

x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_fit_test, sourcePos_x=x_source, sourcePos_y=y_source, 
                                                                 min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
print('image positions: ', x_image, y_image)

-0.6357586484092801 1.2911562510216532
image positions:  [-2.9092635   1.22854335  0.45445759 -0.72416427] [ 3.9677893   4.16077645  4.45908202 -2.03132016]


In [63]:
x_image, y_image = x_image_F200LP[1], y_image_F200LP[1]

alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_fit_test)

x_source, y_source = x_image-alpha_x, y_image-alpha_y

print(x_source, y_source)

x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_fit_test, sourcePos_x=x_source, sourcePos_y=y_source, 
                                                                 min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
print('image positions: ', x_image, y_image)

-0.5115122973124159 1.074229053237786
image positions:  [ 2.74568741 -2.94660606 -1.39863111 -0.95780316] [ 2.71796078  3.63933242  4.31216957 -2.22154292]


In [64]:
x_image, y_image = x_image_F200LP[2], y_image_F200LP[2]

alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_fit_test)

x_source, y_source = x_image-alpha_x, y_image-alpha_y

print(x_source, y_source)

x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_fit_test, sourcePos_x=x_source, sourcePos_y=y_source, 
                                                                 min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
print('image positions: ', x_image, y_image)

-0.7855964022416688 1.2575719689789162
image positions:  [-3.62910879 -0.5067458 ] [ 3.47106652 -2.06127057]


In [65]:
x_image, y_image = x_image_F200LP[3], y_image_F200LP[3]

alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_fit_test)

x_source, y_source = x_image-alpha_x, y_image-alpha_y

print(x_source, y_source)

x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_fit_test, sourcePos_x=x_source, sourcePos_y=y_source, 
                                                                 min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
print('image positions: ', x_image, y_image)

-0.5539130040319626 1.3160609926527744
image positions:  [ 1.91805336 -0.83444981] [ 3.80257696 -1.99585775]


In [66]:
# # DCLS0854-0424

# # x_source_1 = -0.5646987053268822 - (2*0.01)
# # y_source_1 = 0.08835888499899273 + (22*0.01)

# # x_source_2 = -0.343 - (53*0.01)
# # y_source_2 = 0.08835888499899273 + (7*0.01)

# # x_source_3 = -0.343 - (87*0.01)
# # y_source_3 = 0.0393 + (25*0.01)

# x_source_4 = -0.5646987053268822
# y_source_4 = 0.08835888499899273

# # print(x_source_1, y_source_1)
# # print(x_source_2, y_source_2)
# # print(x_source_3, y_source_3)

# kwargs_pemd = {'theta_E': 2.404166080467353,
#                'gamma': 1.8233280499535798,
#                'e1': -0.059740982360995704,
#                'e2': 0.1342377290890493,
#                'center_x': 0.014621001392852512,
#                'center_y': 0.08923949467809739}
# kwargs_shear = {'gamma_ext': 0.10897252162554853,
#                 'psi_ext': -0.81856514273551,
#                 'ra_0': 0,
#                 'dec_0': 0}

# kwargs_lens_list = [kwargs_pemd, kwargs_shear]

# # x_image_1, y_image_1 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_1, sourcePos_y=y_source_1, 
# #                                                                  min_distance=0.01, search_window=5, precision_limit=10**(-10), num_iter_max=100)
# # print('image 1 positions: ', x_image_1, y_image_1)

# # x_image_2, y_image_2 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_2, sourcePos_y=y_source_2, 
# #                                                                  min_distance=0.01, search_window=5, precision_limit=10**(-10), num_iter_max=100)
# # print('image 2 positions: ', x_image_2, y_image_2)

# # x_image_3, y_image_3 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_3, sourcePos_y=y_source_3, 
# #                                                                  min_distance=0.01, search_window=5, precision_limit=10**(-10), num_iter_max=100)
# # print('image 3 positions: ', x_image_3, y_image_3)

# x_image_4, y_image_4 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_4, sourcePos_y=y_source_4, 
#                                                                  min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
# print('image 4 positions: ', x_image_4, y_image_4)

In [67]:
# x_image, y_image = -2.754930458103047, -0.5003156080121595

# alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_lens_list)

# x_source, y_source = x_image-alpha_x, y_image-alpha_y

# print(x_source, y_source)

# x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source, sourcePos_y=y_source, 
#                                                                  min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
# print('image positions: ', x_image, y_image)

F140W Info:

Lens center: 0.6491 0.1451
Image 1, 2 RA: 2.1056694526151 -2.1446663684802947
Image 1, 2 DEC: 0.16656821131187893 -0.09666993968892257
theta E estimate: 2.129037341147402

In [68]:
# x_image, y_image known values
# x_image_F140W = [2.1056694526151, -2.1446663684802947]
# y_image_F140W = [0.16656821131187893, -0.09666993968892257]

# print('F140W image positions: ', x_image_F140W)

F200LP Info:

Lens center: 0.6455 0.1236
Image 1, 2 RA: 2.097939804907363 -2.1223084092251288
Image 1, 2 DEC: 0.13923809099998685 -0.09555221154370663
theta E estimate: 2.1132025459004162

In [69]:
# x_image, y_image known values
# x_image_F200LP = [2.097939804907363, -2.1223084092251288]
# y_image_F200LP = [0.13923809099998685, -0.09555221154370663]

# print('F200LP image positions: ', x_image_F200LP)

In [70]:
# fit the same lens model parameterization to the 4 image positions (free SPEMD model)
# the initial guess of the model can be rather different but 'gamma' has to be kept fixed
# kwargs_lens_init = copy.deepcopy(kwargs_lens_list)
# 'theta_E': 2.311432821867558, 'e1': -0.18785019961075974, 'e2': -0.12607809088893723
# kwargs_lens_init_F140W = [{'theta_E': 2.129037341147402, 'gamma': 2., 'e1': -0.1581155373378608, 'e2': -0.10765765074677205, 'center_x': 0.6491, 'center_y': 0.1451}]

# kwargs_fit_F140W, precision_F140W = solver2Point.constraint_lensmodel(x_pos=x_image_F140W, y_pos=y_image_F140W, 
#                                                           kwargs_list=kwargs_lens_init_F140W, xtol=1.49012e-10)

# kwargs_lens_init_F200LP = [{'theta_E': 2.1132025459004162, 'gamma': 2., 'e1': -0.19999967644529903, 'e2':  0.0258232185418328, 'center_x': 0.6455, 'center_y': 0.1236}]

# kwargs_fit_F200LP, precision_F200LP = solver2Point.constraint_lensmodel(x_pos=x_image_F200LP, y_pos=y_image_F200LP, 
#                                                           kwargs_list=kwargs_lens_init_F200LP, xtol=1.49012e-10)

In [71]:
# x_source_new_F140W, y_source_new_F140W = lensModel.ray_shooting(x_image_F140W, y_image_F140W, kwargs_fit_F140W)
# print('F140W source center:', x_source_new_F140W, y_source_new_F140W)

# x_source_new_F200LP, y_source_new_F200LP = lensModel.ray_shooting(x_image_F200LP, y_image_F200LP, kwargs_fit_F200LP)
# print('F200LP source center:', x_source_new_F200LP, y_source_new_F200LP)

In [72]:
# kwargs_fit_F140W

In [73]:
# kwargs_fit_F200LP

## some plots

In [ ]:
# # make a pixel grid suited for plotting
# from lenstronomy.Util import util



# x_grid, y_grid = util.make_grid(numPix=500, deltapix=0.01)

# # plot convergence

# kappa_grid = lensModel.kappa(x_grid, y_grid, kwargs_fit_F140W)
# kappa2d_fit = util.array2image(kappa_grid)


# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(np.log10(kappa2d_fit), origin='lower')
# ax.text(1, 1, 'convergence, F140W')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)

# plt.show()

# # plot magnification

# mag_grid = lensModel.magnification(x_grid, y_grid, kwargs_fit_F140W)
# mag2d_fit = util.array2image(mag_grid)


# import matplotlib.pyplot as plt
# %matplotlib inline
# from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable

# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(mag2d_fit, origin='lower', vmin=-10, vmax=10)
# ax.text(1, 1, 're-fit model with solver and substructure, F140W')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)


# plt.show()




In [ ]:
# # make a pixel grid suited for plotting
# from lenstronomy.Util import util



# x_grid, y_grid = util.make_grid(numPix=500, deltapix=0.01)

# # plot convergence

# kappa_grid = lensModel.kappa(x_grid, y_grid, kwargs_fit_F200LP)
# kappa2d_fit = util.array2image(kappa_grid)


# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(np.log10(kappa2d_fit), origin='lower')
# ax.text(1, 1, 'convergence, F200LP')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)

# plt.show()

# # plot magnification

# mag_grid = lensModel.magnification(x_grid, y_grid, kwargs_fit_F200LP)
# mag2d_fit = util.array2image(mag_grid)


# import matplotlib.pyplot as plt
# %matplotlib inline
# from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable

# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(mag2d_fit, origin='lower', vmin=-10, vmax=10)
# ax.text(1, 1, 're-fit model with solver and substructure, F200LP')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)


# plt.show()


